In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', clean_up_tokenization_spaces = True)
model = AutoModelForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)  # Adjust num_labels as per your task

print("Model and Tokenizer loaded.")

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and Tokenizer loaded.


In [2]:
from datasets import Dataset
import json
import pandas as pd

reward = "compre" # relate, factual, compre
data_path = f'/kaggle/input/images/{reward}_error1.json'
# Load the dataset
with open(data_path) as f:
    dataset = json.load(f)

# Convert to DataFrame
df = pd.DataFrame.from_dict(dataset, orient='index')
# Inspect the DataFrame
print(df.head())
# df = df.head(1000)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['combined', 'label']])  # Select relevant columns

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['combined'], 
        max_length=1024,
        truncation=True, 
        padding='max_length',
        return_tensors='pt'  # Return PyTorch tensors
    )

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)


                                            combined  label
0  **Text:**\nTime is fast, and I have been buyin...      0
1  **Text:**\nIs the fuel consumption of the down...      1
2  **Text:**\nI usually love to see the informati...      1
3  **Text:**\nStill adhere to the way of "the gol...      1
4  **Text:**\nUnconsciously, the four -wheel driv...      1


Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

In [3]:
# Manually split the dataset into train and validation sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.001)  # Split the entire dataset (90% train, 10% validation)

# Now you can access train and validation sets
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']
print(len(train_dataset))
print(len(val_dataset))
print("split done")

1796
2
split done


In [4]:
from transformers import Trainer, TrainingArguments

import os
os.environ["WANDB_MODE"] = "disabled"

# Set up training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/results',
    eval_strategy='epoch',
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

# bd91daf536563581acb815565de7dc1e2482c2df

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Initializing global attention on CLS token...
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,7.700913
2,1.300300,7.222042
3,1.409700,0.000000
4,0.565200,0.000000
5,0.417000,0.000000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=2245, training_loss=0.8734093205169474, metrics={'train_runtime': 2967.075, 'train_samples_per_second': 3.027, 'train_steps_per_second': 0.757, 'total_flos': 5898527472599040.0, 'train_loss': 0.8734093205169474, 'epoch': 5.0})

In [5]:
results = trainer.evaluate()
print(results)


{'eval_loss': 5.9604641222676946e-08, 'eval_runtime': 0.2638, 'eval_samples_per_second': 7.581, 'eval_steps_per_second': 3.791, 'epoch': 5.0}


In [6]:
# Save the model and tokenizer
save_model_dir = f"/kaggle/working/{reward}"
model.save_pretrained(save_model_dir)
tokenizer.save_pretrained(save_model_dir)


('/kaggle/working/compre/tokenizer_config.json',
 '/kaggle/working/compre/special_tokens_map.json',
 '/kaggle/working/compre/vocab.json',
 '/kaggle/working/compre/merges.txt',
 '/kaggle/working/compre/added_tokens.json',
 '/kaggle/working/compre/tokenizer.json')

In [7]:
import shutil
# destination_dir = "/kaggle/working"
# Zip the model directory
shutil.make_archive(save_model_dir, 'zip', save_model_dir)

'/kaggle/working/compre.zip'